In [10]:
import boto3
import struct
import re
import subprocess
import json
from shapely.geometry import box, mapping

In [11]:
mfa_token = '793813' # replace with latest token
cmd = (f'C:\\Program Files\\Amazon\\AWSCLIV2\\aws sts get-session-token --serial-number arn:aws:iam::693903849513:mfa/Lavender_AWS_MFA  --token-code {mfa_token}').split()
result = subprocess.run(cmd, capture_output=True)
bucket_name = 'frontiersi-p127-floor-height-private'

# Check if the command was successful
if result.returncode == 0:
    # Parse JSON output into a dictionary
    data = json.loads(result.stdout)
    #print(data)  # Print or process the dictionary
else:
    print("Error:", result.stderr)

In [12]:
session = boto3.session.Session(
    aws_access_key_id=data['Credentials']['AccessKeyId'],
    aws_secret_access_key=data['Credentials']['SecretAccessKey'],
    aws_session_token=data['Credentials']['SessionToken']
)
# Initialize an S3 client
s3 = session.client('s3')

In [13]:
# Read LAS header and extract bounding box info
def read_las_header_from_s3(bucket, key):
    try:
        byte_range = 'bytes=0-374'  # LAS header is typically within the first 375 bytes
        response = s3.get_object(Bucket=bucket, Key=key, Range=byte_range)
        header = response['Body'].read()

        # Extract bounding box (min/max X and Y)
        min_x, max_x = struct.unpack('<dd', header[179:195])
        min_y, max_y = struct.unpack('<dd', header[195:211])
        min_z, max_z = struct.unpack('<dd', header[211:227])
        point_count = struct.unpack('<I', header[247:251])[0]  # Total points

        # Fallback: If point count is zero, download the full header file
        if point_count == 0:
            response = s3.get_object(Bucket=bucket, Key=key)
            full_header = response['Body'].read(375)  # Read full header from complete file
            point_count = struct.unpack('<I', full_header[247:251])[0]
        return {
            'min_x': min_x,
            'max_x': max_x,
            'min_y': min_y,
            'max_y': max_y,
            'min_z': min_z,
            'max_z': max_z,
            'point_count': point_count
        }
    except Exception as e:
        print(f"Failed to read {key}: {str(e)}")
        return None
# Convert bounding box to GeoJSON feature
def create_geojson_feature(tile_info, filename):
    bbox = box(tile_info['min_x'], tile_info['min_y'], tile_info['max_x'], tile_info['max_y'])
    feature = {
        'type': 'Feature',
        'geometry': mapping(bbox),
        'properties': {
            'filename': filename,
            'point_count': tile_info['point_count'],
            'min_z': tile_info['min_z'],
            'max_z': tile_info['max_z'],
            'crs': 'EPSG:28355'  # mga94 utm zone 55
        }
    }
    return feature

# Process all LAS files and generate GeoJSON
def generate_geojson_from_s3(bucket_name, prefix, out_file):
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    features = []
    for page in page_iterator:
        for obj in page.get('Contents', []):
            if obj['Key'].endswith('.las'):
                tile_info = read_las_header_from_s3(bucket_name, obj['Key'])
                if tile_info:
                    feature = create_geojson_feature(tile_info, obj['Key'])
                    features.append(feature)

    geojson = {
        'type': 'FeatureCollection',
        'crs': {
            'type': 'name',
            'properties': {
                'name': 'EPSG:28355'  # mga94 utm zone 55
            }
        },
        'features': features
    }

    # Save GeoJSON to file
    with open(out_file, 'w') as f:
        json.dump(geojson, f, indent=4)

    print("GeoJSON file generated: ",out_file)

In [14]:
bucket_name = 'frontiersi-p127-floor-height-private'
prefix = 'lidar/qa4mobile/'
out_file=r'C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\QA4lidar\tile_bbox.geojson'
generate_geojson_from_s3(bucket_name, prefix,out_file)

GeoJSON file generated:  C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\QA4lidar\tile_bbox.geojson
